# Spiking Autoencoders

# 1.1 Shallow Random Sparse LIF, No Inhibition, No Plasticity

Logic:
* Have two populations $\vec{x}$ and $\vec{y}$
* They are connected bi-directionally to each other, but not laterally.
* There is upstream input $\vec{i}^{UP}$, and downstream input $\vec{i}^{DOWN}$

$$ \begin{eqnarray}
  \dot{y}_i &=& -\frac{y_i}{\tau} + W_{ij} x_j + i_i^{DOWN}\\
  \dot{x}_i &=& -\frac{x_i}{\tau} + U_{ij} y_j + i_i^{UP}
\end{eqnarray}$$

* Each unit will spike when it exceeds a standard threshold $T_0$ and be reset to zero.

Test:
* Produce a few patterns
* Present each pattern to $\vec{i}^{UP}$ via poisson process for extended periods of time
* Expect to observe convergence of system to fixed rates
* Measure rates of $\vec{y}$
* Input poisson process proportional to the observed rates to $\vec{i}^{DOWN}$
* Wait for convergence
* Compare rates of $\vec{x}$ to the original pattern


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from brian2 import start_scope, prefs, run
from brian2 import NeuronGroup, PoissonGroup, Synapses, SpikeMonitor, StateMonitor, TimedArray
from brian2 import ms, mV, Hz

# Import libraries from outside folder
import sys
sys.path.append('../../lib/')

import brian2wrapper
import traces_lib

In [4]:
%%time
p = {
    "N_X"       : 100,      # Number of neurons
    "N_Y"       : 100,      # Number of neurons
    "p_conn"    : 0.1,      # Connection probability
    "T0"        : 1.0,      # Average threshold
    "LIF_V_TAU" : 10*ms,    # Neuron leak timescale
    "LIF_V_0"   : 0.0*mV,   # Neuron base voltage
    "LIF_T_0"   : 50.0*mV,  # Neuron spike threshold
    
    "N_PT"      : 4,        # Number of patterns
    "P_PT"      : 0.3,       # Expected proportion of neurons that encode each pattern
    
    "INP_RATE"  : 200*Hz    # Input rate
}

#######################################
# Generate patterns
#######################################

patterns = [(np.random.uniform(0, 1, p['N_X']) < p['P_PT']).astype(int) for i in range(p['N_PT'])]

print('Generated', p['N_PT'], 'patterns')
print('Pattern-neurons:', [np.sum(pt) for pt in patterns])
print('Pattern-overlaps:', [np.sum(patterns[i]+patterns[j] == 2) for i in range(p['N_PT']) for j in range(i+1, p['N_PT'])])

DV_SPIKE_INP = p['LIF_T_0'] / 5                                # Voltage increase per input spike
DV_SPIKE_X   = p['LIF_T_0'] / (1.0 * p['N_X'] * p['p_conn'])   # Voltage increase per system spike
DV_SPIKE_Y   = p['LIF_T_0'] / (1.0 * p['N_Y'] * p['p_conn'])   # Voltage increase per system spike

print("typical potential change per input spike", DV_SPIKE_INP)
print("typical potential change per X spike", DV_SPIKE_X)
print("typical potential change per Y spike", DV_SPIKE_Y)

#######################################
# Brian2 classes
#######################################

start_scope()
prefs.codegen.target = "numpy"

# Create neuronal populations
#G_INP = PoissonGroup(N_EXC, INPUT_FREQ_0 + INPUT_FREQ_MAX * np.ndarray.flatten(img1))
G_X   = brian2wrapper.NeuronGroupLIF(p['N_X'], p['LIF_V_0'], p['LIF_T_0'], p['LIF_V_TAU'])
G_Y   = brian2wrapper.NeuronGroupLIF(p['N_Y'], p['LIF_V_0'], p['LIF_T_0'], p['LIF_V_TAU'])
G_X_INP = PoissonGroup(p['N_X'], p['INP_RATE'] * patterns[0])

# Create synapses
S_XY = Synapses(G_X, G_Y, on_pre='v_post += DV_SPIKE_X', method='exact')
S_YX = Synapses(G_Y, G_X, on_pre='v_post += DV_SPIKE_Y', method='exact')
S_X_INP = Synapses(G_X_INP, G_X, on_pre='v_post += DV_SPIKE_INP', method='exact')

# Connect synapses:
S_X_INP.connect(j='i')
S_XY.connect(p=p['p_conn'])
S_YX.connect(p=p['p_conn'])

spikemon_X = SpikeMonitor(G_X)
spikemon_Y = SpikeMonitor(G_Y)

#######################################
# Run sim
#######################################
RUN_TIME = 2000*ms

run(RUN_TIME)

Generated 4 patterns
Pattern-neurons: [33, 40, 32, 30]
Pattern-overlaps: [10, 12, 6, 13, 14, 10]
typical potential change per input spike 10. mV
typical potential change per X spike 5. mV
typical potential change per Y spike 5. mV
CPU times: user 2.33 s, sys: 12.1 ms, total: 2.34 s
Wall time: 2.34 s


In [ ]:
# TODO:
# 1) Plot average rates for X and Y
# 2) Plot 1-norm between pattern and max_normalized rate of X


spike_idxs = np.array(spikemon_exc.i)
spike_times = np.array(spikemon_exc.t)

spikesPerNeuron = traces_lib.spikes2lists(spike_idxs, spike_times, p['N'])
avgRatePerNeuron = np.array([len(sp) / RUN_TIME for sp in spikesPerNeuron]).astype(float)
avgNetworkRate_t, avgNetworkRate_r = traces_lib.spikes2rate(spike_times, RUN_TIME, 10*ms)
# n1_t, n1_r = traces_lib.spikes2rate(spikesPerNeuron[0], RUN_TIME, 10*ms)
# n2_t, n2_r = traces_lib.spikes2rate(spikesPerNeuron[1], RUN_TIME, 10*ms)
# n3_t, n3_r = traces_lib.spikes2rate(spikesPerNeuron[2], RUN_TIME, 10*ms)

fig, ax = plt.subplots(ncols=3, figsize=(15, 5))
ax[0].plot(np.sort(avgRatePerNeuron))
ax[1].plot(avgNetworkRate_t, avgNetworkRate_r)
ax[2].plot(spikesPerNeuron[0], np.ones(len(spikesPerNeuron[0])), '.')
ax[2].plot(spikesPerNeuron[1], np.ones(len(spikesPerNeuron[1])) + 1, '.')
ax[2].plot(spikesPerNeuron[2], np.ones(len(spikesPerNeuron[2])) + 2, '.')
ax[1].set_xlim([0, float(RUN_TIME)])
ax[2].set_xlim([0, float(RUN_TIME)])
ax[2].set_ylim([0, 3.5])
ax[0].set_title("Time-average activity, per neuron, sorted")
ax[1].set_title("Neuron-average activity, per time")
ax[2].set_title("Example neuron activity, per time")
plt.show()